# LKW 87/186

In [2]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

## Funde

In [3]:
sql = """SELECT
           t_Obj.objID, 
           t_Ort.ort_kurz,
           t_Komplex.bef_nr, 
           t_Obj.Individuum, 
           t_Obj.Typ, 
           t_Obj.Anzahl,  
           t_Obj.Gewicht,
           t_Obj.Gr_Clist, 
           t_Obj.Art,
           t_Obj.Tiefe 
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) Like '%LKW%')
           AND ((t_Komplex.bef_nr) = '87/186'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df['Gewicht'] = df['Gewicht'].convert_objects(convert_numeric=True)
df

,objID,ort_kurz,bef_nr,Individuum,Typ,Anzahl,Gewicht,Gr_Clist,Art,Tiefe
0,205,LKW 186,87/186,"-1:3,13",NeolLeo,1,1448,500,K,
1,1440,LKW 186,87/186,6,,1,16,70,K,
2,1441,LKW 186,87/186,-1:1,NeolLeo,1,56,120,K,
3,1442,LKW 186,87/186,"-1:2,5",NeolLeo,1,92,120,K,
4,2085,LKW 186,87/186,2,NeolLeo,1,45,70,K,
5,2086,LKW 186,87/186,3,NeolLeo,1,39,70,K,
6,2087,LKW 186,87/186,1,,1,60,120,K,
7,2088,LKW 186,87/186,-1:6,,1,155,120,K,
8,2089,LKW 186,87/186,5,PIKMUN (?),1,11,70,K,
9,2090,LKW 186,87/186,-1:7,,1,4,30,K,


In [4]:
df.pivot_table(values = 'Gewicht', index = 'Art', aggfunc = sum, margins = True)

Art
K        1975
Stein     613
All      2588
Name: Gewicht, dtype: float64

In [5]:
df.pivot_table(values = 'objID', index = 'Typ', aggfunc = len, margins = True)

Typ
              13
NeolLeo        5
PIKMUN (?)     1
All           19
Name: objID, dtype: float64

In [6]:
df_pivot1 = df.pivot_table(values = 'Gewicht', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot1 = pd.DataFrame(df_pivot1)
# df_pivot1 = df_pivot1.reset_index()
df_pivot1.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot1['\%'] = (df_pivot1['Gewicht']/df_pivot1['Gewicht'].sum()*100).round(1)
df_pivot1['Gewicht'] = (df_pivot1['Gewicht'] / 1000).round(2)
df_pivot1.rename(columns={'Gewicht': 'Gewicht (kg)'}, inplace=True)

df_pivot2 = df.pivot_table(values = 'Anzahl', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot2 = pd.DataFrame(df_pivot2)
df_pivot2.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot2['\%'] = (df_pivot2['Anzahl']/df_pivot2['Anzahl'].sum()*100).round(1)

# Tabellen vereinen:
df_concat = pd.concat([df_pivot2, df_pivot1], axis = 1)
df_concat = df_concat.reset_index()
df_concat.rename(columns={'Art': 'Fundkategorie'}, inplace=True)
df_concat.to_latex('../output/tabs/9-19_LKW87-186_Funde.tex', index = False, escape = False)
df_concat

,Fundkategorie,Anzahl,\%,Gewicht (kg),\%
0,Keramik,18,94.7,1.98,76.3
1,Stein,1,5.3,0.61,23.7
